In [ ]:
!pip install Pillow

In [ ]:
import pandas as pd
import numpy as np
import cv2
import time
import os
import subprocess
import io
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
from multiprocessing import Pool
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from IPython import display
from matplotlib import animation
from matplotlib.patches import Rectangle
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

In [ ]:
!git clone https://github.com/interactionlab/CapFingerId.git

Cloning into 'CapFingerId'...
remote: Enumerating objects: 219, done.
remote: Total 219 (delta 0), reused 0 (delta 0), pack-reused 219
Receiving objects: 100% (219/219), 130.79 MiB | 12.14 MiB/s, done.
Resolving deltas: 100% (6/6), done.
Updating files: 100% (205/205), done.


In [ ]:
df = ""
PATH = "/content/CapFingerId/data/"

for filename in os.listdir(PATH):
    if filename.endswith(".txt"):
        dfTemp = pd.read_csv(PATH + filename, header=None, names=["Participant", "Handedness", "Finger", "Task", "Ignore", "MatrixStr"], delimiter=";")
        if (type(df) is str):
            df = dfTemp
        else:
            df = pd.concat([df, dfTemp])
    else:
        continue


# Remove the "Ignore" column
df = df.drop('Ignore', axis = 1)

In [ ]:
df.tail()

,Participant,Handedness,Finger,Task,MatrixStr
4173,2,left,thumb,PAUSE,"1508925130,166883971,1,1,2,3,2,4,3,3,4,4,4,4,4..."
4174,2,left,thumb,PAUSE,"1508925130,236762721,1,1,2,3,2,4,3,3,4,4,4,4,4..."
4175,2,left,thumb,PAUSE,"1508925130,236762721,1,1,2,3,2,4,3,3,4,4,4,4,4..."
4176,2,left,thumb,PAUSE,"1508925130,310180533,1,1,2,3,2,4,3,3,4,4,4,181..."
4177,2,left,thumb,PAUSE,"1508925130,377176836,1,1,2,3,2,4,3,3,4,4,4,181..."


In [ ]:
df = df[~df['Participant'].isin(range(3,4))]

In [ ]:
df = df[(df.Finger == "index") | (df.Finger == "thumb")].copy()

In [ ]:
len(df)

114515

In [ ]:
df.Finger.unique()

array(['ring', 'middle', 'little', 'index', 'thumb'], dtype=object)

In [1]:
#Drop all "PAUSE" data
df = df[df.Task != "PAUSE"]

NameError: name 'df' is not defined

In [ ]:
%%time
temp_var = None
def transform_to_matrix(s):
    matrix = s.replace("\n", "")
    matrix = matrix.split(",")

    if (len(matrix) != 408):
        return -1, -1

    matrix = matrix[:407]

    # determine timestamp
    timestamp = str(matrix[0]) + str(matrix[1][:3])
    timestamp = int(timestamp)

    matrix = np.array(matrix[2:]).reshape(27, 15)

    matrix[matrix==''] = '0'
    matrix[matrix=='-'] = '0'

    try:
        matrix = matrix.astype(int)
    except ValueError:
        return -1, -1

    return matrix, timestamp

df['Matrix'] = df.MatrixStr.apply(lambda x : transform_to_matrix(x)[0])
df['Timestamp'] = df.MatrixStr.apply(lambda x : transform_to_matrix(x)[1])
df = df.drop(['MatrixStr'], axis=1)
df = df[~(df.Timestamp == -1)]

CPU times: user 1min 4s, sys: 1.28 s, total: 1min 5s
Wall time: 1min 5s


In [ ]:
THRESHOLD = 30

# Recursive blob detection
def blob_detection(matrix, x, y, found):
    if x > 0 and x < len(matrix[0]) and y > 0 and y < len(matrix) and matrix[y][x] > THRESHOLD and ((x,y) not in found):
        found.append((x, y))
        blob_detection(matrix, x + 1, y, found) # right
        blob_detection(matrix, x - 1, y, found) # left
        blob_detection(matrix, x, y + 1, found) # top
        blob_detection(matrix, x, y - 1, found) # down

# Returns Blob Coordinates, connected tiles > 30
def get_blobs(matrix):
    blobs = []
    founds = []
    for y in range(0, len(matrix)):
        for x in range(0, len(matrix[0])):

            # blob detection
            found = []
            blob_detection(matrix, x, y, found)

            if (len(found) > 0):
                sorted_by_x = sorted(found, key=lambda x: x[0])
                sorted_by_y = sorted(found, key=lambda x: x[1])
                x_min = sorted_by_x[0][0]
                y_min = sorted_by_y[0][1]
                x_max = sorted_by_x[len(sorted_by_x) - 1][0]
                y_max = sorted_by_y[len(sorted_by_y) - 1][1]

                if (x_min - 1, x_max + 1, y_min - 1, y_max + 1) not in blobs and ((x_max - x_min) * (y_max - y_min)) > 1:
                    blobs.append((x_min - 1, x_max + 1, y_min - 1, y_max + 1))
                    founds.append(len(found))

    return blobs

# Returns the cutted blob
def get_blob_contents(matrix, blob_coordinates):
    matrices = []
    for c in blob_coordinates:
        matrices.append(matrix[c[2]:c[3], c[0]:c[1]])
    return matrices


# Returns the coordinates of the rectangle containing the touch blob
def get_plt_rect(blob_coordinates):
    return Rectangle((blob_coordinates[2] - 0.5, blob_coordinates[0]  - 0.5),
     (blob_coordinates[1] - blob_coordinates[0]) + 1, (blob_coordinates[3] - blob_coordinates[2]) + 1,
                     fill=None, alpha=1, lw=2, color='lime')

In [ ]:
%%time
df['BlobCoords'] = df.Matrix.apply(lambda x : get_blobs(x))
df['BlobCount'] = df.BlobCoords.apply(lambda x : len(x))
df['Blobs'] = [get_blob_contents(df.Matrix.iloc[i], df.BlobCoords.iloc[i]) for i in range(len(df))]

# remove all empty images and images with more than one blob
df = df[df.BlobCount == 1]

CPU times: user 1min 16s, sys: 191 ms, total: 1min 16s
Wall time: 1min 17s


In [ ]:
df['Blobs'] = df['Blobs'].apply(lambda x: np.array(x))
df['Blobs'] = df['Blobs'].apply(lambda x: np.squeeze(np.array(x)))

In [ ]:
def delete_negative(blobs):
    blobs = np.where(blobs < 0, 0, blobs)
    return blobs
df['Blobs']=df.Blobs.apply(lambda x: delete_negative(x))

In [ ]:
#Resize with OpenCV dùng bilinear interpolation
import cv2
import numpy as np

def resize_blob_with_opencv(blob):
    blob_array = blob

    blob_array = blob_array.astype(np.float64)

    #Resize to 8x8 using bilinear interpolation
    resized_blob = cv2.resize(blob_array, (8, 8), interpolation=cv2.INTER_LINEAR)

    return resized_blob

df['Resized_Blobs'] = df['Blobs'].apply(resize_blob_with_opencv)

In [ ]:
# Tìm giá trị nhỏ nhất trong tất cả các ma trận trong cột 'BlobImage'
min_value = df['Scaled_Blobs'].apply(lambda x: x.max()).max()
min_value

255

#Scale up using Lanczos algorithm

In [ ]:
# Define the size factor for scaling up
scale_factor = 5
def upscale_image(blob, scale_factor):
    # Create a PIL image from the numpy array
    image = Image.fromarray(blob.astype(np.uint8), 'L')

    # Upscale the image using the Lanczos resampling algorithm
    upscaled_image = image.resize(
        (image.size[0] * scale_factor, image.size[1] * scale_factor), Image.LANCZOS
    )

    # Convert the upscaled PIL image to a numpy array for further use
    return np.array(upscaled_image)

# Now apply this function to each item in the 'Blobs' column
df['Scaled_Blobs'] = df['Blobs'].apply(lambda x: upscale_image(x, scale_factor))

# Adaptive Threshold to find Ellipse

In [ ]:
# Hàm tính diện tích của elip
def calculate_area(a, b):
    return math.pi * a * b

# Khởi tạo biến count bên ngoài hàm để theo dõi số lần không tìm thấy contour
count = 0

def find_ellipse_params(img):
    global count

    # Áp dụng adaptive thresholding
    thresh = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)

    # Tìm contour trong ảnh nhị phân
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Nếu không tìm thấy contour
    if not contours:
        count += 1  # Tăng biến count lên 1

    # Chọn contour lớn nhất
    largest_contour = max(contours, key=cv2.contourArea)

    # Tìm elip bao xung quanh contour lớn nhất
    ellipse = cv2.fitEllipse(largest_contour)

    # Trả về các thông số của elip
    major_axis = ellipse[1][0]
    minor_axis = ellipse[1][1]
    area = calculate_area(major_axis / 2, minor_axis / 2)
    angle = ellipse[2]

    return major_axis, minor_axis, area, angle
# Tạo các cột mới trong DataFrame để lưu thông tin của elip
df['Major_Axis'] = np.nan
df['Minor_Axis'] = np.nan
df['Area'] = np.nan
df['Angle'] = np.nan

# Duyệt qua toàn bộ DataFrame
for index, row in df.iterrows():
    # Xác định các thông số của elip cho mỗi ảnh trong cột 'Scaled_Blobs'
    major_axis, minor_axis, area, angle = find_ellipse_params(row['Scaled_Blobs'])
    # Lưu các thông số vào DataFrame
    df.at[index, 'Major_Axis'] = major_axis
    df.at[index, 'Minor_Axis'] = minor_axis
    df.at[index, 'Area'] = area
    df.at[index, 'Angle'] = angle

In [ ]:
df.head()

,Participant,Handedness,Finger,Task,Matrix,Timestamp,BlobCoords,BlobCount,Blobs,BlobImage,BlobImgFlattened,Scaled_Blobs,Major_Axis,Minor_Axis,Area,Angle
26,7,right,index,SCROLL,"[[0, -1, 0, -3, 0, 0, 2, 1, 0, 1, -2, 2, 0, 1,...",1509965139675,"[(9, 13, 11, 16)]",1,"[[4, 9, 13, 5], [5, 31, 57, 15], [13, 120, 220...","[[4, 9, 13, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[4, 9, 13, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[3, 4, 4, 5, 5, 6, 7, 9, 11, 11, 12, 13, 12, ...",9.804294,14.024021,107.988808,42.690639
27,7,right,index,SCROLL,"[[0, -1, 0, -3, 0, 0, 2, 1, 0, 1, -2, 2, 0, 1,...",1509965139675,"[(9, 13, 11, 16)]",1,"[[4, 9, 13, 5], [5, 31, 57, 15], [13, 120, 220...","[[4, 9, 13, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[4, 9, 13, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[3, 4, 4, 5, 5, 6, 7, 9, 11, 11, 12, 13, 12, ...",10.345570,12.890965,104.744140,47.160843
28,7,right,index,SCROLL,"[[0, -1, 0, -3, 0, 0, 2, 1, 0, 1, -2, 2, 0, 1,...",1509965139745,"[(8, 11, 11, 15)]",1,"[[4, 14, 12], [14, 78, 83], [18, 124, 140], [9...","[[4, 14, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[4, 14, 12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[[2, 2, 2, 3, 2, 4, 4, 4, 4, 2, 3, 2, 1, 0, 0]...",10.345570,12.890965,104.744140,47.160843
29,7,right,index,SCROLL,"[[0, -1, 0, -3, 0, 0, 2, 1, 0, 1, -2, 2, 0, 1,...",1509965139817,"[(6, 10, 10, 14)]",1,"[[6, 8, 15, 9], [15, 46, 69, 13], [23, 151, 23...","[[6, 8, 15, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[6, 8, 15, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[[5, 5, 5, 5, 4, 4, 5, 6, 7, 9, 11, 12, 13, 12...",9.906662,13.497671,105.020981,54.622498
30,7,right,index,SCROLL,"[[0, -1, 0, -3, 0, 0, 2, 1, 0, 1, -2, 2, 0, 1,...",1509965139885,"[(4, 8, 10, 14)]",1,"[[5, 11, 23, 9], [9, 50, 155, 52], [13, 86, 23...","[[5, 11, 23, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[5, 11, 23, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[[4, 5, 4, 4, 5, 6, 5, 5, 4, 4, 3, 2, 2, 2, 2,...",8.511100,14.011619,93.662096,77.459496


# Save Data

In [ ]:
df.to_pickle('/content/drive/MyDrive/FingerClassification/Data/minidata.pkl')